In [ ]:
import numpy as np
import psdr
from scipy.linalg import orth
from psdr.opg import opg_grads
from psdr.perplexity_opg import perplexity_opg_grads

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Build a test function
np.random.seed(0)
m = 5
Q = orth(np.random.randn(m,m))
lam = np.zeros(m)
lam = np.linspace(1,10, m)**(-2)
A = Q @ np.diag(lam) @ Q.T
f =  lambda x: 0.5 * x.T @ A @ x
grad = lambda x: A @ x
fun = psdr.Function(f, psdr.BoxDomain(-np.ones(m), np.ones(m)), grads = [grad])

## Example: Random Data

In [ ]:
X = fun.domain.sample(2000)
fX = fun(X)
grads_true = fun.grad(X)

print("OPG")
%time grads1 = opg_grads(X, fX)
print("Perplexity OPG")
%time grads2 = perplexity_opg_grads(X, fX)
err1 = np.max(np.abs(grads1 - grads_true), axis = 1)
err2 = np.max(np.abs(grads2 - grads_true), axis = 1)

fig, ax = plt.subplots()
xx = np.linspace(0, max(np.max(err1), np.max(err2)), 50)
ax.hist(err1, xx, alpha = 0.5, label = 'opg')
ax.hist(err2, xx, alpha = 0.5, label = 'popg');
ax.set_xlabel('error, inf-norm')
ax.set_ylabel('density')
ax.legend();

## Example: Mixed Data
In this example, data is drawn from two distributions to reflect the challenges inherent when we have data that doesn't emerge from a simple random sample approach.

In [ ]:
X1 = fun.domain.sample(1000)
X2 = 0.1*fun.domain.sample(1000)
X = np.vstack([X1,X2])
fX = fun(X)
grads_true = fun.grad(X)

print("OPG")
%time grads1 = opg_grads(X, fX)
print("Perplexity OPG")
%time grads2 = perplexity_opg_grads(X, fX)
err1 = np.max(np.abs(grads1 - grads_true), axis = 1)
err2 = np.max(np.abs(grads2 - grads_true), axis = 1)

fig, ax = plt.subplots()
xx = np.linspace(0, max(np.max(err1), np.max(err2)), 100)
ax.hist(err1, xx, alpha = 0.5, label = 'opg')
ax.hist(err2, xx, alpha = 0.5, label = 'popg');
ax.set_xlabel('error, inf-norm')
ax.set_ylabel('density')
ax.legend();